# L4: Support Data Insight Analysis

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
#from helper import load_env

from dotenv import load_dotenv
load_dotenv("/home/jovyan/Albert/.env")

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [6]:
cd /home/jovyan/Dongping/crewai/L4

/home/jovyan/Dongping/crewai/L4


## Loading Tasks and Agents YAML files

In [7]:
# Define file paths for YAML configurations
files = {
    'agents': '/home/jovyan/Dongping/crewai/L4/agents.yaml',
    'tasks': '/home/jovyan/Dongping/crewai/L4/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [8]:
configs

{'agents': {'suggestion_generation_agent': {'role': 'Suggestion Engine\n',
   'goal': 'Generate actionable suggestions for resolving issues identified in the support tickets, leveraging historical data and predefined rules.\n',
   'backstory': 'You specialize in analyzing past resolutions and current issues to provide tailored suggestions that can help the support team resolve tickets efficiently.\n',
   'verbose': True,
   'allow_delegation': False},
  'reporting_agent': {'role': 'Report Generator\n',
   'goal': 'Compile a summary report that includes issue classification results, suggested actions, and key trends observed in the support data.\n',
   'backstory': 'You are skilled at transforming raw data into insightful reports that help stakeholders understand trends and make informed decisions.\n',
   'verbose': True,
   'allow_delegation': False},
  'chart_generation_agent': {'role': 'Chart Specialist\n',
   'goal': 'Create visual representations of the data provided by the Reporti

## Using FileReadTool

In [9]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

In [10]:
csv_tool

FileReadTool(name="Read a file's content", description="Tool Name: Read a file's content\nTool Arguments: {}\nTool Description: A tool that can be used to read ./support_tickets_data.csv's content.", args_schema=<class 'crewai_tools.tools.file_read_tool.file_read_tool.FixedFileReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7ff6c588d620>, result_as_answer=False, file_path='./support_tickets_data.csv')

In [11]:
import pandas as pd
df=pd.read_csv('./support_tickets_data.csv')
df.head()

,ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,resolved
0,T0001,C0511,API Issue,I'm pleased with how my issue was handled. Tha...,High,2023-03-25,240,927,4,I'm pleased with how my issue was handled. Tha...,A004,True
1,T0002,C0729,Login Issue,Excellent service! The agent went above and be...,Low,2023-04-06,223,534,5,The problem still persists. Not resolved yet.,A004,False
2,T0003,C0439,Report Generation,Resolution was satisfactory but could be impro...,Low,2023-04-25,214,592,1,"The issue was escalated quickly, which was app...",A004,False
3,T0004,C0100,Data Import,Agent was very helpful and polite. Great service!,High,2023-02-20,110,864,2,The issue was resolved quickly. Very satisfied!,A003,True
4,T0005,C0258,Feature Request,"My issue was handled well, but follow-up could...",Medium,2023-01-18,151,193,2,The problem still persists. Not resolved yet.,A004,False


In [12]:
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

## Creating Agents, Tasks and Crew

In [13]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

#chart_generation_agent = Agent(
#  config=agents_config['chart_generation_agent'],
#  allow_code_execution=True
#)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

#chart_generation = Task(
#  config=tasks_config['chart_generation'],
#  agent=chart_generation_agent
#)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation]
  #context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    final_report_assembly
  ],
  verbose=True
)


## Testing our Crew

In [16]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

2024-12-02 09:34:03,864 - 140698965987392 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2024-12-02 09:34:04,073 - 140698965987392 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.APIError: APIError: OpenAIException - Invalid response object Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/litellm_core_utils/llm_response_utils/convert_dict_to_response.py", line 380, in convert_to_model_response_object
    assert response_object["choices"] is not None and isinstance(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError


received_args={'response_object': {'id': None, 'choices': None, 'created': None, 'model': None, 'object': None, 'service_tier': None, 'system_fingerprint': None, 'usage': None, 'llm-gateway-requestid': '4261fbc526114e70991d8f911c3e62b6', 'responseCode': 501, 'responseDesc': 'LLM Request Failed', 'Error': "Error code: 404 - {'erro

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the s

2024-12-02 09:34:04,199 - 140698965987392 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.APIError: APIError: OpenAIException - Invalid response object Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/litellm_core_utils/llm_response_utils/convert_dict_to_response.py", line 380, in convert_to_model_response_object
    assert response_object["choices"] is not None and isinstance(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 860, in completion
    raise e
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 811, in completion
    return convert_to_model_response_object(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



APIError: litellm.APIError: APIError: OpenAIException - Invalid response object Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/litellm_core_utils/llm_response_utils/convert_dict_to_response.py", line 380, in convert_to_model_response_object
    assert response_object["choices"] is not None and isinstance(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 860, in completion
    raise e
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/llms/OpenAI/openai.py", line 811, in completion
    return convert_to_model_response_object(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/envs/llm_agent_crewai/lib/python3.11/site-packages/litellm/l

## Training your crew and agents

In [ ]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

## Comparing new test results

In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

## Kicking off Crew

In [ ]:
result = support_report_crew.kickoff()

## Result

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))